## Webscraping Danish News sites

This is a webscraper to gather news from the most popular Danish media:  
- TV2 Nyheder  
- BT Nyheder  
- DR
- Ekstrabladet  

The structure of the sites are abit different, but the overall method is:  
- Navigate to news front page, accept cookies and expand dynamic content to get a fair amount of recent news.
- Identify all real articles and corresponding links.  
- Visit each link and extract article text along with the title.  
- Create dataframe with all news.  
  

I use Selenium with Chrome to navigate dynamic content and then Requests and BeautifulSoup for the actual extraction.

In [1]:
# Import libraries
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import datetime
import os
from tqdm.notebook import tqdm_notebook
import time

pd.options.display.max_colwidth = 500

In [ ]:
# Open Chrome
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

### TV2 Nyheder

In [3]:
driver.get('https://nyheder.tv2.dk')

In [4]:
# Accepting cookies
ignored_exceptions=(NoSuchElementException,StaleElementReferenceException,)
WebDriverWait(driver, 10,ignored_exceptions=ignored_exceptions)\
                        .until(EC.presence_of_element_located((By.XPATH,'//button[@class="sc-ifAKCX dvvOSu"]'))).click()

In [5]:
# Expanding dynamic content by clicking "Load more" - increase range for more news
time.sleep(10)
for i in range(5):
    time.sleep(1)
    ActionChains(driver).move_to_element(WebDriverWait(driver, 10,ignored_exceptions=ignored_exceptions)\
                                         .until(EC.element_to_be_clickable((By.XPATH, '//div[@class="tc_load-more js-tc_load-more__context"]')))).click().perform()

In [6]:
html = driver.page_source
# Convert to soup object
soup = BeautifulSoup(html)

In [7]:
# Generate links
links = []
for l in soup.find_all('article'):
    links.append(l.find('a')['href'])

# Exclude LIVE
links = links[1:]

In [8]:
page = requests.get(links[5])
soup = BeautifulSoup(page.content, "html.parser")
soup_p = soup.find("div", class_="tc_page__body")
soup_p2 = soup.find("h1")

In [9]:
# Extract content from article links
content_dict = {}
titles_dict = {}

# Loop through all article pages
for l in tqdm_notebook(range(len(links))):

    try:
        page = requests.get(links[l])
        soup = BeautifulSoup(page.content, "html.parser")
        soup_p = soup.find("div", class_="tc_page__body")
        soup_p2 = soup.find("h1")

        content = soup_p.text
        titles = soup_p2.text

        content_dict[l] = content
        titles_dict[l] = titles
    # Ignore errors on items that are not real articles - quick'n'dirty style
    except (AttributeError):
        pass

  0%|          | 0/82 [00:00<?, ?it/s]

In [10]:
# Create dataframe with articles
links_df = pd.DataFrame(links,columns=['link'])
titles_df = pd.DataFrame.from_dict(titles_dict,orient='index',columns=['title']).reset_index()
content_df = pd.DataFrame.from_dict(content_dict,orient='index',columns=['content']).reset_index()
articles_tv2_df = pd.concat([titles_df,content_df,links_df],axis=1,join='inner').drop(['index'],axis=1)

In [11]:
articles_tv2_df['medie'] = 'TV2'

### BT Nyheder

In [12]:
driver.get('https://www.bt.dk/nyheder')

In [13]:
# Accepting cookies
driver.implicitly_wait(10)
driver.find_element(By.ID,'CybotCookiebotDialogBodyLevelButtonLevelOptinAllowAll').click()

In [14]:
# Expanding dynamic content by clicking "Load more"
for i in range(5):
    driver.find_element(By.XPATH,'//a[contains(text(), "Indlæs\n                flere")]').click()

In [15]:
# Extract HTML
driver.implicitly_wait(10)
html = driver.page_source
# Convert to soup object
soup = BeautifulSoup(html)

In [16]:
# Generate links
links = []
for l in soup.find_all('article',class_="teaser teaser--flat cat-news"):
    links.append(l.find('a')['data-lpurl'])

# Exclude LIVE
links = links[1:]

In [17]:
# Extract content
content_dict = {}
titles_dict = {}

# Loop through all article pages
for l in tqdm_notebook(range(len(links))):
    
    try:
        page = requests.get(links[l])
        soup = BeautifulSoup(page.content, "html.parser")
        soup_p = soup.find("div", class_="article-content")
        soup_p2 = soup.find("h1", class_="article-title")

        content = soup_p.text
        titles = soup_p2.text

        # Clean text
        content = content.replace('\n',' ')
        content = re.sub(r" +"," ",content)
        content_dict[l] = content

        titles = titles.replace('\n',' ')
        titles = re.sub(r" +"," ",titles)
        titles_dict[l] = titles

    # Ignore errors on items that are not real articles - quick'n'dirty style
    except (AttributeError):
        pass

  0%|          | 0/52 [00:00<?, ?it/s]

In [18]:
# Create dataframe with articles
links_df = pd.DataFrame(links,columns=['link'])
titles_df = pd.DataFrame.from_dict(titles_dict,orient='index',columns=['title']).reset_index()
content_df = pd.DataFrame.from_dict(content_dict,orient='index',columns=['content']).reset_index()
articles_bt_df = pd.concat([titles_df,content_df,links_df],axis=1).drop(['index'],axis=1)

In [19]:
articles_bt_df['medie'] = 'BT'

### DR Nyheder

In [20]:
driver.get('https://www.dr.dk/nyheder')

In [21]:
# Accepting cookies
driver.implicitly_wait(20)
driver.find_element(By.XPATH, "//button[@class='drcc-button submitAll']").click()

In [22]:
# Extract HTML
driver.implicitly_wait(10)
html = driver.page_source
# Convert to soup object
soup = BeautifulSoup(html)

In [23]:
# Generate links
links = []
for l in soup.find_all('div',class_='dre-teaser'):
    links.append('https://dr.dk/'+l.find('a')['href'])

In [24]:
# Extract content
content_list = []
titles_list = []

titles = soup.find_all('h2', class_="hydra-latest-news-page-short-news__title dre-variables")
content = soup.find_all("article", class_="hydra-latest-news-page-short-news")   

for i in tqdm_notebook(range(len(titles))):
    content_list.append(content[i].text)
    titles_list.append(titles[i].text)

  0%|          | 0/50 [00:00<?, ?it/s]

In [25]:
# Create dataframe with articles
links_df = pd.DataFrame(links,columns=['link'])
titles_df = pd.DataFrame(titles_list,columns=['title'])
content_df = pd.DataFrame(content_list,columns=['content'])
articles_dr_df = pd.concat([links_df,titles_df,content_df],axis=1)

In [26]:
articles_dr_df['medie'] = 'DR'

### Ekstrabladet

I use the front page instead of the actual news page because there are many paid articles on the news page.

In [27]:
# Go to site
driver.get("https://ekstrabladet.dk/")

In [28]:
# Accepting cookies
driver.implicitly_wait(20)
driver.find_element(By.XPATH, "//button[@id='CybotCookiebotDialogBodyLevelButtonLevelOptinAllowAll']").click()

In [29]:
# Extract HTML
driver.implicitly_wait(10)
html = driver.page_source
# Convert to soup object
soup = BeautifulSoup(html)

In [30]:
# Find articles
articles = soup.find_all("article")   

In [31]:
# Only keep non-paid articles
non_paid = []
for a in articles:
    try:
        if a['data-priority'] != 'A+':
            non_paid.append(a)
    except KeyError as ke:
        print('not an article')

not an article


In [32]:
# Generate links
links = []
for l in non_paid:
    links.append('https://ekstrabladet.dk'+l.find("a")['href'])

In [33]:
# Extract content
content_dict = {}
titles_dict = {}

# Loop through all article pages
for l in tqdm_notebook(range(len(links))):
    page = requests.get(links[l])
    soup = BeautifulSoup(page.content, "html.parser")
    soup_p = soup.find("div", class_="article-bodytext")
    soup_p2 = soup.find("h1", class_="art-title")

    # Some links are video articles without text, quick'n'dirty ignore
    try:
        content = soup_p.text
        titles = soup_p2.text

        # Clean text
        content = content.replace('\n',' ')
        content = content.replace('--------- SPLIT ELEMENT ---------',' ')
        content = content.replace('\xa0',' ')
        content = re.sub(r" +"," ",content)
        content_dict[l] = content

        titles = titles.replace('\n',' ')
        titles = re.sub(r" +"," ",titles)
        titles_dict[l] = titles

    except (AttributeError, NameError):
        pass


  0%|          | 0/34 [00:00<?, ?it/s]

In [34]:
# Create dataframe with articles
links_df = pd.DataFrame(links,columns=['link'])
titles_df = pd.DataFrame.from_dict(titles_dict,orient='index',columns=['title']).reset_index()
content_df = pd.DataFrame.from_dict(content_dict,orient='index',columns=['content']).reset_index()
articles_eb_df = pd.concat([links_df,titles_df,content_df],axis=1,join='inner').drop(['index'],axis=1)

In [35]:
articles_eb_df['medie'] = 'EB'

### Concatenation and save

In [36]:
articles_df = pd.concat([articles_dr_df,articles_eb_df,articles_tv2_df,articles_bt_df]).reset_index().drop(['index'],axis=1)

In [37]:
articles_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211 entries, 0 to 210
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   link     211 non-null    object
 1   title    211 non-null    object
 2   content  211 non-null    object
 3   medie    211 non-null    object
dtypes: object(4)
memory usage: 6.7+ KB


In [38]:
articles_df.head()

,link,title,content,medie
0,https://dr.dk//nyheder/seneste/pollensaesonen-er-blevet-laengere-og-vaerre-de-sidste-ti-aar,Pollensæsonen er blevet længere og værre de sidste ti år,"18 min. sidenPollensæsonen er blevet længere og værre de sidste ti årLæs opOrdbogTekstAf Frederik PalleNys, kløe og lidt problemer med vejrtrækningen. En stor del af den danske befolkning har de samme problemer i denne periode, og det kan være, at du føler problemerne længere end normalt.Det er der nemlig en god grund til. Den udfordrende pollensæson er nemlig længere, end den tidligere har været. Det viser nye tal fra Astma-Allergi Danmarks pollenkalender ifølge Politiken.De sidste ti år er...",DR
1,https://dr.dk//nyheder/seneste/se-billederne-kvinde-protesterer-paa-roed-loeber-i-cannes-med-det-ukrainske-flag,Se billederne: Kvinde protesterer på rød løber i Cannes med det ukrainske flag malet på kroppen,"59 min. sidenSe billederne: Kvinde protesterer på rød løber i Cannes med det ukrainske flag malet på kroppenSikkerhedsfolk måtte stoppe kvinden, der løb ind på den røde løber til filmfestivalen i Cannes. (Foto: VALERY Hache © Ritzau Scanpix)Læs opOrdbogTekstAf Frederik Palle""Stop med at voldtage os!""Det var teksten, der stod skrevet på kvindens krop, da hun i dag løb delvist nøgen ind på den røde løber ved filmfestivalen i Cannes.Udover teksten havde hun også malet det ukrainske flag på sin ...",DR
2,https://dr.dk//nyheder/seneste/rusland-paastaar-staalvaerk-i-mariupol-er-blevet-fuldstaendigt-befriet,"Rusland påstår, at stålværk i Mariupol er 'blevet fuldstændigt befriet'","I dag kl. 21:18Rusland påstår, at stålværk i Mariupol er 'blevet fuldstændigt befriet'Læs opOrdbogTekstAf Maja Lærke MaachIfølge det russiske forsvarsministerium har de sidste ukrainske soldater, der havde forskanset sig på Azovstal-værket, overgivet sig.Området er 'blevet fuldstændigt befriet', lyder det fra ministeriet, skriver Reuters. Angiveligt har 531 ukrainske soldater overgivet sig.Ukraine har ikke bekræftet udmeldingen fra russerne, men den ukrainske præsident er her til aften citer...",DR
3,https://dr.dk//nyheder/seneste/tornadoer-i-det-vestlige-tyskland-kvaester-40-personer,To tornadoer i det vestlige Tyskland kvæster 40 personer,"I dag kl. 21:02To tornadoer i det vestlige Tyskland kvæster 40 personerLæs opOrdbogTekstAf Frederik PalleTo tornadoer har ramt det vestlige Tyskland, og de har sat et stort aftryk. Blandt andet er der 40 personer, som er kvæstet i byen Paderborn. Det skriver tyske Bildzeitung ifølge Ritzau.Ti personer af de 40 er alvorligt sårede.Politiet i Paderborn opfordrer borgere til at blive hjemme. Den offentlige transport er hårdt ramt.Byen Lippstadt er også blandt de mærkede byer, hvor en tornador r...",DR
4,https://dr.dk//sporten/seneste-sport/atp-reagerer-paa-russisk-udelukkelse-ingen-ranglistepoint-ved-wimbledon,Tennisforbund reagerer på russisk udelukkelse: Ingen ranglistepoint ved Wimbledon,"I dag kl. 20:24Tennisforbund reagerer på russisk udelukkelse: Ingen ranglistepoint ved WimbledonDen prestigefyldte grand slam-turnering løber fra den 27. juni til og med den 10. juli. (Foto: pool © Ritzau Scanpix)Læs opOrdbogTekstDer bliver ingen point til verdensranglisten til mandlige deltagere ved årets Wimbledon.Det meddeler Det Internationale Tennisforbund (ATP) på sin hjemmeside fredag.Beslutningen skyldes Wimbledons udelukkelse af russiske og belarusiske spillere, som ATP erklærede si...",DR


In [50]:
articles_df['medie'].value_counts()

TV2    75
BT     52
DR     50
EB     34
Name: medie, dtype: int64

In [39]:
def save_df(yn,df,path):
    dd = str(datetime.date.today())
    if not os.path.exists(path):
        os.mkdir(path)
    if yn == True:
        df.to_csv('{}/articles_{}.csv'.format(path,dd),index=False,encoding='utf-8')
        print('Saved')
    else:
        print('Not saved')      

In [41]:
save_df(True,articles_df,r'D:\OneDrive\')

Saved
